In [1]:
from openai import OpenAI
from dotenv import dotenv_values
import json
import math
import numpy as np
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

import functions.prompts as prompts

config = dotenv_values(".env")

client = OpenAI(
    api_key=config['DEEPINFRA_TOKEN'],
    base_url="https://api.deepinfra.com/v1/openai",
)

In [2]:
def gen_body(text, model):
    return {
            "model": model,
            "messages":[
                {
                    "role": "user",
                    "content": f"Generate {text} paper that could be published in a top-5 economics journal. Start the output with paper title without any pretext"
                }],
            "max_completion_tokens": 64000
        }

In [ ]:
model = "google/gemma-3-27b-it"
model_name = "gemma"

fields = ['a finance', 'a macroeconomics', 'a microeconimics', 'an applied microeconomics']
reqs = [[ {
        "id": f'{field.split(" ")[1][:3]}_{i}',
        "body": gen_body(field, model=model)
    } for i in range(5) ] for field in fields]

reqs = [ x for xs in reqs for x in xs ]

def gen_paper(client, req):
    print(f"Generating: {req['id']}")

    paper = client.chat.completions.create(**req['body'])
    text = paper.choices[0].message.content
    
    with open(f"./fake-papers/{model_name}-{req['id']}.txt", "w") as f:
        f.write(text)

In [4]:
with ThreadPoolExecutor(max_workers=200) as executor:
    results = list(executor.map(
        gen_paper, 
        [client] * len(reqs),
        reqs,
    ))

Generating: fin_0
Generating: fin_1
Generating: fin_2
Generating: fin_3
Generating: fin_4
Generating: mac_0
Generating: mac_1
Generating: mac_2
Generating: mac_3
Generating: mac_4
Generating: mic_0
Generating: mic_1
Generating: mic_2
Generating: mic_3
Generating: mic_4
Generating: app_0
Generating: app_1
Generating: app_2
Generating: app_3
Generating: app_4
